<a href="https://colab.research.google.com/github/nazzang49/boost-camp-projects/blob/main/assignments/(P06)GPT_LM_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 학습해보기

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
1기 멘토
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
2기 멘토
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
이녕우 (leenw2@gmail.com)
박채훈 (qkrcogns2222@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

이번 시간엔 한국어 코퍼스를 활용해, 직접 한국어 GPT-2를 학습해보겠습니다.

In [ ]:
!pip install transformers

In [ ]:
import torch
torch.cuda.is_available()

True

역시 위키 데이터를 가져와볼까요?

In [ ]:
!mkdir my_data
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1728      0 --:--:-- --:--:-- --:--:--  1728
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0  2456k      0 --:--:-- --:--:-- --:--:-- 2456k


In [ ]:
# 한국어 위키 데이터
path = "/content/my_data/wiki_20190620_small.txt"

지금까지는 BertWordPieceTokenizer를 사용해왔다면,   
이번에는 SentencePiceBPETokenizer를 사용해 모델을 학습해보겠습니다.

각 tokenizer의 차이는 허훈님의 블로그 [여기](https://huffon.github.io/2020/07/05/tokenizers/) 에서 확인하실 수 있습니다.



In [ ]:
from tokenizers import SentencePieceBPETokenizer
from tokenizers.normalizers import BertNormalizer

tokenizer = SentencePieceBPETokenizer()

tokenizer._tokenizer.normalizer = BertNormalizer(clean_text=True,
handle_chinese_chars=False,
lowercase=False)

# GPT 모델은 단방향으로 문맥을 이해하기 때문에 [SEP] 불필요
tokenizer.train(
    path,
    vocab_size=10000,
    special_tokens=[
        "<s>",   # 문장 시작
        "<pad>",
        "</s>",  # 문장 종료
        "<unk>",
    ],
)



In [ ]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))
# SentencePiece를 사용하면, 나중에 decoding 과정에서 '_' 만 ' '로 replace해주면 띄어쓰기 복원이 가능해집니다.

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[1005, 577, 6613, 1303, 1041, 2071, 1136, 594, 1033]
['▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.']
이순신은 조선 중기의 무신이다.


In [ ]:
tokenizer.save_model(".")

['./vocab.json', './merges.txt']

In [ ]:
# vocab 파일 기반 tokenizer init
tokenizer = SentencePieceBPETokenizer.from_file(vocab_filename="vocab.json", merges_filename="merges.txt")

In [ ]:
vocab_list = list(tokenizer.get_vocab().keys())

In [66]:
# path = 'vocab.txt'
# with open(path, 'wb') as f:
#     pickle.dump(vocab_list, f)

path = 'vocab.txt'
with open(path, 'w') as f:
    tmp = ''
    for v in vocab_list:
        tmp += (v + '\n')
    f.writelines(tmp)

In [ ]:
path = 'vocab.txt'
with open(path, 'rb') as f:
    v = pickle.load(f)
    print(v)

['▁그런데', '관리', '▁유일하게', '▁사업을', '임이', '%의', '▁점과', '▁저작권', '▁서울특별', '▁베스핀', '림', '▁신행정수', '동안', '름-리우', '▁중심에', '▁처음으로', '훌', '릴', '상이', '셴코', '인먼', '▁소프트웨어,', '-ROM', '에도', '▁엔트로피의', '▁데', '▁산맥', '음에', '▁사용에', '스케', '프', '▁프랑스', '디빌', '▁가족과', '국통일', '정식', '버스터', '어진', '인지', '▁터미', '▁멈', '크래프트', '▁22', '하는데', '맛', '리지', '▁동형', '▁경찰은', '▁관리,', '와는', '사람', '▁옹호', '력의', '스는', '군은', '▁1세기', '▁것이라든', '▁엑', '▁표현할', '▁용', '▁맥스웰은', '▁의사결', '▁삼국', '▁방정식이', '4년', '▁띠', '40', '날레', '▁열린집합', '▁넘는', '▁사람이', '▁훨', '▁아래와', '▁C를', '기에', '▁뇌물', '▁스위스', '▁기술이', '▁구성할', '▁위치를', '거래', '▁도발', '▁중앙', '▁공식은', '감을', '▁청계', '▁영토', '▁불이', '▁마크업', '▁송', '▁실려', '▁리스트', '시대', '▁대형', '▁화강', '▁머물', '▁의식', '사건,', '대학교에서', '▁심사', '▁집', '▁존재를', '셀', '▁법칙과', '인다.', '선에서', '▁정권의', '▁업적', '립', '’', '▁정부는', '▁공간은', '▁대입', '슈타인', '▁진정', '▁작품은', '▁제국이', '▁아래의', '▁직후', '▁값이', '멜', '웹', '▁간단', '돈', '▁공의', '▁데이터베이', '▁컬', '▁직관', '▁10만', '▁소련', '펴', '몽', '이었으나', '다"라며', '수함', '▁금속', '▁원자의', '▁과정을', '▁formula_2에', '▁공집합이', '▁원주율을

In [ ]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[1005, 577, 6613, 1303, 1041, 2071, 1136, 594, 1033]
['▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.']
['▁<', 's', '>', '이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.', '<', '/s', '>']
<s>이순신은 조선 중기의 무신이다.</s>


In [ ]:
# 명시적인 special token 추가
tokenizer.add_special_tokens(["<s>", "</s>", "<unk>", "<pad>", "<shkim>"])
tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")
tokenizer.unk_token_id = tokenizer.token_to_id("<unk>")
tokenizer.bos_token_id = tokenizer.token_to_id("<bos>")
tokenizer.eos_token_id = tokenizer.token_to_id("<eos>")

print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

[0, 1005, 577, 6613, 1303, 1041, 2071, 1136, 594, 1033, 2]
['<s>', '▁이', '순', '신은', '▁조선', '▁중', '기의', '▁무', '신', '이다.', '</s>']
이순신은 조선 중기의 무신이다.


In [ ]:
from transformers import GPT2Config, GPT2LMHeadModel

# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.get_vocab_size(),
  bos_token_id=tokenizer.token_to_id("<s>"),
  eos_token_id=tokenizer.token_to_id("</s>"),
)

# creating the model
model = GPT2LMHeadModel(config)

In [ ]:
model.num_parameters()

93523200

In [ ]:
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from torch.utils.data.dataset import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging

logger = logging.get_logger(__name__)

class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(is_pair=False)

        # 캐싱
        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                # 여기서부터 본격적으로 데이터셋을 만들기 시작합니다.
                logger.info(f"Creating features from dataset file at {directory}")
                self.examples = []
                text = ""
                with open(file_path, encoding="utf-8") as f:
                    lines = f.readlines()
                    for line in lines:
                        line = line.strip()
                        line = "<s>"+line+"</s>" # 학습 데이터 앞 뒤에 문장 구분 기호를 추가해줍니다.
                        text += line    # 'text' 객체에 모든 학습 데이터를 다 합쳐버립니다 :-)
                tokenized_text = tokenizer.encode(text).ids

                # 모델의 최대 sequence length (=block_size) 만큼 데이터를 잘라서 저장합니다.
                for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    self.examples.append(
                        tokenized_text[i : i + block_size]
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [ ]:
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=path,
    block_size=128,
)
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없습니다 :-)
    tokenizer=tokenizer, mlm=False,
)

In [ ]:
print(dataset[0])

tensor([   0, 3997, 3545, 8404,  463,    4, 5480, 9527, 1798, 1890, 2297, 1262,
        9625, 2679, 1188, 2174,    2,    0, 5708, 5480,  254, 6466,  749, 3425,
         872, 1556,  679,  894, 1627, 9223,  586, 3620, 1010, 3302,    2,    0,
        6466, 7417, 2305,  403, 2217, 1074,    2,    0, 1013, 1107, 3715,  645,
        8574, 1024,  939,   92, 7322,  371,   92,  720, 9295,  704, 1651,  453,
        3167, 1032, 1074,    2,    0, 6343, 1262, 3715, 1009, 2932, 1176,  912,
        2037, 1171, 3228,  843,   92,  439,  973, 1486, 1017,    3, 1323, 3914,
        2095, 1042,    2,    0, 1382, 2068, 2225, 1095,  326,  843, 1823,  506,
           4, 1240, 7697,    2,    0, 3897, 6466, 1053, 1077,  685, 2318, 4649,
        5204, 5671, 1013, 1759,  114, 2742, 3004,  103,  654, 2283, 9764, 1192,
        1796, 2449, 2546, 9937, 6466, 1053, 1037,  533])


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=64, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1897
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1500


Step,Training Loss
100,7.933900
200,7.397600
300,7.053700
400,6.720400
500,6.444100
600,6.213200
700,6.007400
800,5.843400
900,5.695800
1000,5.570800


Saving model checkpoint to model_output/checkpoint-1000
Configuration saved in model_output/checkpoint-1000/config.json
Model weights saved in model_output/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1500, training_loss=6.105784016927084, metrics={'train_runtime': 1288.17, 'train_samples_per_second': 73.632, 'train_steps_per_second': 1.164, 'total_flos': 6195887308800000.0, 'train_loss': 6.105784016927084, 'epoch': 50.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to model_output
Configuration saved in model_output/config.json
Model weights saved in model_output/pytorch_model.bin


In [ ]:
USE_GPU = 1
# Device configuration
device = torch.device('cuda' if (torch.cuda.is_available() and USE_GPU) else 'cpu')

In [ ]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("<s>이순신", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

# input_ids 기반 3개 문장 생성
output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


GENERATED SEQUENCE : 이순신이 매우 것은 발표하는 것은 한다.
GENERATED SEQUENCE : 이순신드에서부터 독일의부터 당시 "·선르들이 맞고 초들은 거공'까지 그는 다했다.
GENERATED SEQUENCE : 이순신 다음과 같은 직의 화학다.


# Huggingface 모델 업로드

In [ ]:
!apt-get install git-lfs

In [ ]:
!transformers-cli login

WARNING! `transformers-cli login` is deprecated and will be removed in v5. Please use `huggingface-cli login` instead.

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: jaywhypark
Password: 
Login successful
Your token: ACixLpyjFhRoEbevFZWbRElfwARLgEtGmFjFtGWiThvPHLSZnFMGrCzGCGwwCXZfeVCgHMxlJLRfxLjcZcPnrhbdjUuWHqXhNQkjFaANFBwUjmuKULfRlMQFVaTztMEZ 

Your token has been saved to /root/.huggingface/token


In [ ]:
!huggingface-cli repo create test

git version 2.17.1
Error: unknown flag: --version

Sorry, no usage text found for "git-lfs"

You are about to create jaywhypark/test
Proceed? [Y/n] y

Your repo now lives at:
  https://huggingface.co/jaywhypark/test

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/jaywhypark/test



In [ ]:
!git clone https://jaywhypark:ACixLpyjFhRoEbevFZWbRElfwARLgEtGmFjFtGWiThvPHLSZnFMGrCzGCGwwCXZfeVCgHMxlJLRfxLjcZcPnrhbdjUuWHqXhNQkjFaANFBwUjmuKULfRlMQFVaTztMEZ@huggingface.co/jaywhypark/test

Cloning into 'test'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (3/3), done.


In [ ]:
!git config --global user.email 'nazzang49@gmail.com'
!git config --global user.name 'jaywhypark'

In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [ ]:
!cd MyDrive/MyDrive/

In [ ]:
# 디렉토리 이동
%cd test

/content/test


In [ ]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [67]:
!git add --all

In [68]:
!git commit -m 'Vocab file has been changed'

[main 5106222] Vocab file has been changed
 1 file changed, 10001 insertions(+)


In [69]:
!git push

Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 39.28 KiB | 5.61 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
To https://huggingface.co/jaywhypark/test
   88895bd..5106222  main -> main


# 업로드 된 모델 확인

In [70]:
from transformers import AutoModel, AutoTokenizer, pipeline

In [ ]:
model = AutoModel.from_pretrained("jaywhypark/test") # 직접 업로드 한 모델
tokenizer = AutoTokenizer.from_pretrained("taeminlee/kogpt2") # 오픈 소스

feature = pipeline('feature-extraction', model= model.to('cpu'), tokenizer= tokenizer)
print(feature)